In [6]:
import random
import math

# Function to parse Solomon instance from file
def parse_solomon_instance(file_path):
    customers = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        # Skip instance name and empty lines
        vehicle_info = lines[4].split()
        num_vehicles = int(vehicle_info[0])  # Max number of vehicles
        capacity = int(vehicle_info[1])      # Vehicle capacity
        
        # Read customer data starting from line 9
        for line in lines[9:]:
            data = line.split()
            if len(data) == 7:
                customer = {
                    'id': int(data[0]),
                    'x': float(data[1]),
                    'y': float(data[2]),
                    'demand': float(data[3]),
                    'ready_time': float(data[4]),
                    'due_date': float(data[5]),
                    'service_time': float(data[6])
                }
                customers.append(customer)
    
    depot = customers[0]  # Customer 0 is the depot
    return customers, depot, capacity, num_vehicles

# Distance calculation
def calculate_distance(c1, c2):
    return math.sqrt((c1['x'] - c2['x'])**2 + (c1['y'] - c2['y'])**2)

# Decode particle position into routes
def decode_position(position, customers, depot, capacity):
    customer_indices = list(range(1, len(customers)))  # Exclude depot (0)
    sorted_indices = sorted(customer_indices, key=lambda i: position[i-1], reverse=True)
    
    routes = []
    current_route = []
    current_capacity = 0
    current_time = 0
    
    for idx in sorted_indices:
        customer = customers[idx]
        if current_capacity + customer['demand'] <= capacity:
            last_node = depot if not current_route else customers[current_route[-1]]
            travel_time = calculate_distance(last_node, customer)
            arrival_time = current_time + travel_time
            
            if arrival_time <= customer['due_date']:
                wait_time = max(0, customer['ready_time'] - arrival_time)
                current_time = max(arrival_time, customer['ready_time']) + customer['service_time']
                current_route.append(idx)
                current_capacity += customer['demand']
            else:
                if current_route:
                    routes.append(current_route)
                current_route = [idx]
                current_capacity = customer['demand']
                current_time = customer['ready_time'] + customer['service_time']
        else:
            if current_route:
                routes.append(current_route)
            current_route = [idx]
            current_capacity = customer['demand']
            current_time = customer['ready_time'] + customer['service_time']
    
    if current_route:
        routes.append(current_route)
    
    # Calculate total distance
    total_distance = 0
    for route in routes:
        prev = depot
        for idx in route:
            total_distance += calculate_distance(prev, customers[idx])
            prev = customers[idx]
        total_distance += calculate_distance(prev, depot)  # Return to depot
    
    return routes, total_distance

# Fitness function
def fitness(routes, total_distance, num_customers):
    num_vehicles = len(routes)
    served_customers = sum(len(route) for route in routes)
    penalty = 10000 * (num_customers - served_customers)  # Penalize unserved customers
    return num_vehicles * 1000 + total_distance + penalty  # Prioritize fewer vehicles

# Particle class
class Particle:
    def __init__(self, num_customers):
        self.position_i = [random.uniform(0, 1) for _ in range(num_customers-1)]  # 100 customers
        self.velocity_i = [random.uniform(-0.1, 0.1) for _ in range(num_customers-1)]
        self.pos_best_i = self.position_i.copy()
        self.err_best_i = float('inf')
        self.err_i = float('inf')

# PSO algorithm
def pso_vrptw(customers, depot, capacity, num_particles, max_iter):
    swarm = [Particle(len(customers)) for _ in range(num_particles)]
    global_best_pos = None
    global_best_err = float('inf')
    
    for iteration in range(max_iter):
        for particle in swarm:
            routes, distance = decode_position(particle.position_i, customers, depot, capacity)
            particle.err_i = fitness(routes, distance, len(customers)-1)
            
            if particle.err_i < particle.err_best_i:
                particle.pos_best_i = particle.position_i.copy()
                particle.err_best_i = particle.err_i
            
            if particle.err_i < global_best_err:
                global_best_pos = particle.position_i.copy()
                global_best_err = particle.err_i
            
            w, c1, c2 = 0.7, 2.0, 2.0
            for d in range(len(particle.position_i)):
                r1, r2 = random.random(), random.random()
                particle.velocity_i[d] = (w * particle.velocity_i[d] +
                                         c1 * r1 * (particle.pos_best_i[d] - particle.position_i[d]) +
                                         c2 * r2 * (global_best_pos[d] - particle.position_i[d]))
                particle.position_i[d] = max(0, min(1, particle.position_i[d] + particle.velocity_i[d]))
    
    routes, distance = decode_position(global_best_pos, customers, depot, capacity)
    return routes, distance, len(routes)

# Main execution with your Solomon file
text_file = 'Data\\c101.txt'
customers, depot, capacity, max_vehicles = parse_solomon_instance(text_file)
print(parse_solomon_instance(text_file))

# Run PSO with 30 particles and 200 iterations
routes, distance, num_vehicles = pso_vrptw(customers, depot, capacity, 30, 200)

# Output results
print(f"Instance: C102")
print(f"Number of Vehicles: {num_vehicles}")
print(f"Total Distance: {distance:.2f}")
print(f"Routes:")
for i, route in enumerate(routes, 1):
    print(f"  Vehicle {i}: {route}")

([{'id': 0, 'x': 40.0, 'y': 50.0, 'demand': 0.0, 'ready_time': 0.0, 'due_date': 1236.0, 'service_time': 0.0}, {'id': 1, 'x': 45.0, 'y': 68.0, 'demand': 10.0, 'ready_time': 912.0, 'due_date': 967.0, 'service_time': 90.0}, {'id': 2, 'x': 45.0, 'y': 70.0, 'demand': 30.0, 'ready_time': 825.0, 'due_date': 870.0, 'service_time': 90.0}, {'id': 3, 'x': 42.0, 'y': 66.0, 'demand': 10.0, 'ready_time': 65.0, 'due_date': 146.0, 'service_time': 90.0}, {'id': 4, 'x': 42.0, 'y': 68.0, 'demand': 10.0, 'ready_time': 727.0, 'due_date': 782.0, 'service_time': 90.0}, {'id': 5, 'x': 42.0, 'y': 65.0, 'demand': 10.0, 'ready_time': 15.0, 'due_date': 67.0, 'service_time': 90.0}, {'id': 6, 'x': 40.0, 'y': 69.0, 'demand': 20.0, 'ready_time': 621.0, 'due_date': 702.0, 'service_time': 90.0}, {'id': 7, 'x': 40.0, 'y': 66.0, 'demand': 20.0, 'ready_time': 170.0, 'due_date': 225.0, 'service_time': 90.0}, {'id': 8, 'x': 38.0, 'y': 68.0, 'demand': 20.0, 'ready_time': 255.0, 'due_date': 324.0, 'service_time': 90.0}, {'id'

In [10]:
import random
import math

# Function to parse Solomon instance from file
def parse_solomon_instance(file_path):
    customers = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        vehicle_info = lines[4].split()
        num_vehicles = int(vehicle_info[0])  # Max number of vehicles
        capacity = int(vehicle_info[1])      # Vehicle capacity
        
        for line in lines[9:]:
            data = line.split()
            if len(data) == 7:
                customer = {
                    'id': int(data[0]),
                    'x': float(data[1]),
                    'y': float(data[2]),
                    'demand': float(data[3]),
                    'ready_time': float(data[4]),
                    'due_date': float(data[5]),
                    'service_time': float(data[6])
                }
                customers.append(customer)
    
    depot = customers[0]  # Customer 0 is the depot
    return customers, depot, capacity, num_vehicles

# Distance calculation
def calculate_distance(c1, c2):
    return math.sqrt((c1['x'] - c2['x'])**2 + (c1['y'] - c2['y'])**2)

# Decode particle position into routes
def decode_position(position, customers, depot, capacity, max_vehicles):
    customer_indices = list(range(1, len(customers)))  # Exclude depot (0)
    sorted_indices = sorted(customer_indices, key=lambda i: position[i-1], reverse=True)
    
    routes = []
    current_route = []
    current_capacity = 0
    current_time = 0
    
    for idx in sorted_indices:
        customer = customers[idx]
        if len(routes) >= max_vehicles and current_route:  # If max vehicles reached, stop adding new routes
            break
        if current_capacity + customer['demand'] <= capacity:
            last_node = depot if not current_route else customers[current_route[-1]]
            travel_time = calculate_distance(last_node, customer)
            arrival_time = current_time + travel_time
            
            if arrival_time <= customer['due_date']:
                wait_time = max(0, customer['ready_time'] - arrival_time)
                current_time = max(arrival_time, customer['ready_time']) + customer['service_time']
                current_route.append(idx)
                current_capacity += customer['demand']
            else:
                if current_route:
                    routes.append(current_route)
                current_route = [idx]
                current_capacity = customer['demand']
                current_time = customer['ready_time'] + customer['service_time']
        else:
            if current_route:
                routes.append(current_route)
            current_route = [idx]
            current_capacity = customer['demand']
            current_time = customer['ready_time'] + customer['service_time']
    
    if current_route and len(routes) < max_vehicles:
        routes.append(current_route)
    
    # Calculate total distance
    total_distance = 0
    for route in routes:
        prev = depot
        for idx in route:
            total_distance += calculate_distance(prev, customers[idx])
            prev = customers[idx]
        total_distance += calculate_distance(prev, depot)  # Return to depot
    
    return routes, total_distance

# Fitness function with vehicle restriction
def fitness(routes, total_distance, num_customers, max_vehicles):
    num_vehicles = len(routes)
    served_customers = sum(len(route) for route in routes)
    penalty_unserved = 10000 * (num_customers - served_customers)  # Penalize unserved customers
    penalty_vehicles = 100000 * max(0, num_vehicles - max_vehicles)  # Penalize exceeding max vehicles
    return num_vehicles * 1000 + total_distance + penalty_unserved + penalty_vehicles

# Particle class
class Particle:
    def __init__(self, num_customers):
        self.position_i = [random.uniform(0, 1) for _ in range(num_customers-1)]
        self.velocity_i = [random.uniform(-0.1, 0.1) for _ in range(num_customers-1)]
        self.pos_best_i = self.position_i.copy()
        self.err_best_i = float('inf')
        self.err_i = float('inf')

# PSO algorithm
def pso_vrptw(customers, depot, capacity, num_particles, max_iter, max_vehicles):
    swarm = [Particle(len(customers)) for _ in range(num_particles)]
    global_best_pos = None
    global_best_err = float('inf')
    
    for iteration in range(max_iter):
        for particle in swarm:
            routes, distance = decode_position(particle.position_i, customers, depot, capacity, max_vehicles)
            particle.err_i = fitness(routes, distance, len(customers)-1, max_vehicles)
            
            if particle.err_i < particle.err_best_i:
                particle.pos_best_i = particle.position_i.copy()
                particle.err_best_i = particle.err_i
            
            if particle.err_i < global_best_err:
                global_best_pos = particle.position_i.copy()
                global_best_err = particle.err_i
            
            w, c1, c2 = 0.7, 2.0, 2.0
            for d in range(len(particle.position_i)):
                r1, r2 = random.random(), random.random()
                particle.velocity_i[d] = (w * particle.velocity_i[d] +
                                         c1 * r1 * (particle.pos_best_i[d] - particle.position_i[d]) +
                                         c2 * r2 * (global_best_pos[d] - particle.position_i[d]))
                particle.position_i[d] = max(0, min(1, particle.position_i[d] + particle.velocity_i[d]))
    
    routes, distance = decode_position(global_best_pos, customers, depot, capacity, max_vehicles)
    return routes, distance, len(routes)

# Main execution
text_file = 'Data\\c101.txt'
customers, depot, capacity, max_vehicles = parse_solomon_instance(text_file)

# Run PSO with 30 particles and 200 iterations
routes, distance, num_vehicles = pso_vrptw(customers, depot, capacity, 30, 1000, max_vehicles)

# Output results
print(f"Instance: c101")
print(f"Maximum Number of Vehicles: {max_vehicles}")
print(f"Number of Vehicles Used: {num_vehicles}")
print(f"Total Distance: {distance:.2f}")
print(f"Routes:")
for i, route in enumerate(routes, 1):
    print(f"  Vehicle {i}: {route}")

Instance: c101
Maximum Number of Vehicles: 25
Number of Vehicles Used: 25
Total Distance: 2782.73
Routes:
  Vehicle 1: [2]
  Vehicle 2: [3, 7, 8, 11, 12]
  Vehicle 3: [13, 14, 26]
  Vehicle 4: [32, 33, 35, 39, 51]
  Vehicle 5: [61]
  Vehicle 6: [63, 64]
  Vehicle 7: [65, 73, 89, 91]
  Vehicle 8: [96, 97, 100]
  Vehicle 9: [81, 21]
  Vehicle 10: [18]
  Vehicle 11: [5, 23, 22, 69]
  Vehicle 12: [95, 9]
  Vehicle 13: [37, 75]
  Vehicle 14: [29, 28, 52, 49]
  Vehicle 15: [93]
  Vehicle 16: [55]
  Vehicle 17: [67, 41, 60, 80]
  Vehicle 18: [25, 15, 79, 99]
  Vehicle 19: [76, 74, 30]
  Vehicle 20: [71, 4, 66]
  Vehicle 21: [34]
  Vehicle 22: [57, 54, 56, 1, 47]
  Vehicle 23: [92, 16, 59]
  Vehicle 24: [24, 31, 53, 84, 77]
  Vehicle 25: [83, 38]
